<a href="https://colab.research.google.com/github/yosuke318/python_notebook/blob/main/%E9%9D%9E%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 遅延処理の例

In [11]:
import asyncio

async def delayed_function(seconds):
    await asyncio.sleep(seconds)
    print(f"{seconds}秒後に実行")

async def main():
    await asyncio.gather(
        delayed_function(3),
        delayed_function(5)
    )

await main()

# 同期関数から非同期関数を呼ぶ

In [39]:
import asyncio
import time

async def async_func():
    print("func: start async_func")
    await asyncio.sleep(1)
    print("func: end async_func")

def main():
    print("main: start main")
    print("main: before func")
    loop = asyncio.get_event_loop()
    future: Future = loop.run_in_executor(None, sync_func)
    print("main: after func")
    time.sleep(3)
    print("main: end main")


main()

main: start main
main: before func
main: after func
func: start sync_func
func: end sync_func
main: end main


# **非同期関数から非同期関数を呼ぶ**


In [31]:
import asyncio
import time


async def async_func():
    print("func: start async_func")
    await asyncio.sleep(1)
    print("func: end async_func")

async def main():
    print("main: start main")
    print("main: before func")
    res = async_func()
    print("main: after func")
    print("main: before await")
    print("main: after await")
    await asyncio.sleep(3)
    print("main: end main")


await main()


main: start main
main: before func
main: after func
main: before await
main: after await
main: end main


<ipython-input-31-fecab63cd421>:21: RuntimeWarning: coroutine 'async_func' was never awaited
  await main()


上記の場合、非同期関数(coroutine)をawiatなしで実行したために、`RuntimeWarning: coroutine async_func was never awaited`という警告が出る。基本的にasync def で定義された関数は別のasync def関数でのみawaitできる。

上記の処理を修正し、coroutineにawaitをつけて実行

In [32]:
import asyncio
import time


async def async_func():
    print("func: start async_func")
    await asyncio.sleep(1)
    print("func: end async_func")

async def main():
    print("main: start main")
    print("main: before func")
    await async_func()
    print("main: after func")
    print("main: before await")
    print("main: after await")
    await asyncio.sleep(3)
    print("main: end main")


await main()

main: start main
main: before func
func: start async_func
func: end async_func
main: after func
main: before await
main: after await
main: end main


# 同期関数の完了を待たずに実行

In [29]:
import asyncio
import time

def sync_func():
  print("func: start sync_func")
  print("func: end sync_func")


async def main():
  print("main: start main")
  print("main: before func")

  loop = asyncio.get_event_loop()
  future: Future = loop.run_in_executor(None, sync_func)

  print("main: after func")
  await asyncio.sleep(3)

  # run_in_executerの戻り値futureをawaitすることでより確実にsync_funcの完了を待つことができる
  # await future
  print("main: end main")

await main()

main: start main
main: before func
main: after funcfunc: start sync_func
func: end sync_func

main: end main


# 非同期関数の呼び出し時点から実行を開始して欲しい場合

ここではcreate_task関数を使って、async_func開始直後に`main: after func`を標準出力に出す

In [42]:
import asyncio
import time

async def async_func():
    print("func: start async_func")
    await asyncio.sleep(1)
    print("func: end async_func")

async def main():
    print("main: start main")
    print("main: before func")
    task = asyncio.create_task(async_func())
    await asyncio.sleep(0)
    print("main: after func")
    await asyncio.sleep(3)
    print("main: end main")


await main()

main: start main
main: before func
func: start async_func
main: after func
func: end async_func
main: end main
